In [1]:
! python --version


Python 3.9.19


In [51]:
import sys
print(sys.executable)

import torch
print(torch.__version__)

H:\Conda\envs\pytorch-env\python.exe
2.2.2+cu121


In [52]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. Here are the GPU details:")
    print(torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")

CUDA is available. Here are the GPU details:
NVIDIA GeForce RTX 3080


In [53]:
import numpy as np
import pandas as pd

In [54]:
df_user_score=pd.read_csv('./users-score-2023.csv')
df_user_score.head()

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
1,1,Xinil,48,.hack//Sign,7
2,1,Xinil,320,A Kite,5
3,1,Xinil,49,Aa! Megami-sama!,8
4,1,Xinil,304,Aa! Megami-sama! Movie,8


In [55]:
df_anime=pd.read_csv('./anime-dataset-2023.csv')
df_anime.head()

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...


In [56]:
# merge the Type col into user_score dataset
merged_dataset = pd.merge(df_user_score, df_anime[['anime_id', 'Type']], on='anime_id', how='left')
merged_dataset.head()

,user_id,Username,anime_id,Anime Title,rating,Type
0,1,Xinil,21,One Piece,9,TV
1,1,Xinil,48,.hack//Sign,7,TV
2,1,Xinil,320,A Kite,5,OVA
3,1,Xinil,49,Aa! Megami-sama!,8,OVA
4,1,Xinil,304,Aa! Megami-sama! Movie,8,Movie


In [57]:
# Grouping the dataset by genre
grouped = merged_dataset.groupby('Type')

# Creating a dictionary of DataFrames for each tpye
type_datasets = {Type: group.copy() for Type, group in grouped}

In [58]:
unique_types = merged_dataset['Type'].unique()
print("unique_type:", unique_types)

unique_type: ['TV' 'OVA' 'Movie' 'Special' 'ONA' 'Music' 'UNKNOWN' nan]


In [59]:
# seperate the dataset based on Type
TV_dataset = type_datasets['TV']
Movie_dataset = type_datasets['Movie']
OVA_dataset = type_datasets['OVA']
ONA_dataset = type_datasets['ONA']
Special_dataset = type_datasets['Special']
Music_dataset = type_datasets['Music'] # weird type

In [60]:
combined_TV_dataset = pd.concat([TV_dataset, OVA_dataset, Special_dataset, ONA_dataset]) # include all episodes into tv category
combined_TV_dataset.head()

,user_id,Username,anime_id,Anime Title,rating,Type
0,1,Xinil,21,One Piece,9,TV
1,1,Xinil,48,.hack//Sign,7,TV
5,1,Xinil,306,Abenobashi Mahou☆Shoutengai,8,TV
6,1,Xinil,53,Ai Yori Aoshi,7,TV
8,1,Xinil,591,Amaenaide yo!!,6,TV


In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

In [62]:
# deal with TV dataset
# shuffle the data since was ordered by the user id
combined_TV_dataset = shuffle(combined_TV_dataset, random_state=66)
# do the encoding
# Create a MinMaxScaler object
scaler = MinMaxScaler(feature_range=(0, 1))
# Scale the 'score' column between 0 and 1
combined_TV_dataset['scaled_score'] = scaler.fit_transform(combined_TV_dataset[['rating']])
user_encoder = LabelEncoder()
anime_encoder = LabelEncoder()
combined_TV_dataset['anime_id_encoded'] = anime_encoder.fit_transform(combined_TV_dataset['anime_id'])
combined_TV_dataset['user_id_encoded'] = user_encoder.fit_transform(combined_TV_dataset['user_id'])
# drop tv col
del combined_TV_dataset['Type']
combined_TV_dataset.head()

,user_id,Username,anime_id,Anime Title,rating,scaled_score,anime_id_encoded,user_id_encoded
12747797,400650,ice-goblin,206,Lodoss-tou Senki: Eiyuu Kishi Den,7,0.666667,169,138153
5375099,83996,Kanamiz_dreamer,330,Midori no Hibi,6,0.555556,285,51954
5353350,83602,Fatalmemory,384,Gantz,7,0.666667,336,51705
395413,4808,Joolee,28223,Death Parade,8,0.777778,7146,2778
2105555,27967,joe-anime,1575,Code Geass: Hangyaku no Lelouch,9,0.888889,1220,18373


In [65]:
# set X and y for tv
# X is the features used for prediction
# y is the target
X_tv = combined_TV_dataset[['user_id_encoded','anime_id_encoded']].values
y_tv = combined_TV_dataset['scaled_score'].values
print("Shape of X_tv:", X_tv.shape)
print("Shape of y_tv:", y_tv.shape)

Shape of X_tv: (20894155, 2)
Shape of y_tv: (20894155,)


In [68]:
# deal with Movie dataset
# shuffle the data since was ordered by the user id
Movie_dataset = shuffle(Movie_dataset, random_state=66)
# do the encoding
# Create a MinMaxScaler object
scaler = MinMaxScaler(feature_range=(0, 1))
# Scale the 'score' column between 0 and 1
Movie_dataset['scaled_score'] = scaler.fit_transform(Movie_dataset[['rating']])
user_encoder = LabelEncoder()
anime_encoder = LabelEncoder()
Movie_dataset['anime_id_encoded'] = anime_encoder.fit_transform(Movie_dataset['anime_id'])
Movie_dataset['user_id_encoded'] = user_encoder.fit_transform(Movie_dataset['user_id'])
# drop Movie col
del Movie_dataset['Type']
Movie_dataset.head()

,user_id,Username,anime_id,Anime Title,rating,scaled_score,anime_id_encoded,user_id_encoded
5344476,83431,spineslayer,543,Vampire Hunter D (2000),9,0.888889,63,37636
23988737,1264223,jeffweed,2847,Pokemon Movie 10: Dialga vs. Palkia vs. Darkrai,8,0.777778,412,180993
9625514,335574,Vulcanite,418,Ranma ½: Chuugoku Nekonron Daikessen! Okite Ya...,7,0.666667,28,71358
16691075,489726,Valentineline,2000,Roujin Z,8,0.777778,255,125820
1925476,25571,Renim,430,Fullmetal Alchemist: The Conqueror of Shamballa,5,0.444444,30,12484


In [69]:
# set X and y for Movie
# X is the features used for prediction
# y is the target
X_movie = Movie_dataset[['user_id_encoded','anime_id_encoded']].values
y_movie = Movie_dataset['scaled_score'].values
print("Shape of X_movie:", X_movie.shape)
print("Shape of y_movie:", y_movie.shape)

Shape of X_movie: (3346546, 2)
Shape of y_movie: (3346546,)


In [73]:
# split training data and validation data for tv dataset
X_tv_train, X_tv_test, y_tv_train, y_tv_test = train_test_split(X_tv, y_tv, test_size=10000/20894155)

X_tv_train = X_tv_train.astype('int32')
X_tv_test = X_tv_test.astype('int32')
print("Number of samples in the training set for tv:", len(y_tv_train))
print("Number of samples in the test set for tv:", len(y_tv_test))

Number of samples in the training set for tv: 20884155
Number of samples in the test set for tv: 10000


In [74]:
# split training data and validation data for movie dataset
X_movie_train, X_movie_test, y_movie_train, y_movie_test = train_test_split(X_movie, y_movie, test_size=10000/3346546)

X_movie_train = X_movie_train.astype('int32')
X_movie_test = X_movie_test.astype('int32')
print("Number of samples in the training set for movie:", len(y_movie_train))
print("Number of samples in the test set for movie:", len(y_movie_test))

Number of samples in the training set for movie: 3336546
Number of samples in the test set for movie: 10000


In [75]:
class CollaborativeFilteringModel(nn.Module):
    def __init__(self, num_users, num_animes, embedding_size):
        super(CollaborativeFilteringModel, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.anime_embedding = nn.Embedding(num_animes, embedding_size)
        self.dense = nn.Linear(1, 1)
        self.sigmoid = nn.Sigmoid()
        self.flatten = nn.Flatten()
        self.activation = nn.Sigmoid()  # or nn.ReLU()

    def forward(self, user_input, anime_input):
        user_embedded = self.user_embedding(user_input)
        anime_embedded = self.anime_embedding(anime_input)
        dot_product = torch.mul(user_embedded, anime_embedded).sum(dim=-1)
        flattened = self.flatten(dot_product.unsqueeze(-1))
        activated = self.activation(flattened)
        dense_output = self.dense(activated)
        output = self.sigmoid(dense_output)
        return output

In [76]:
num_users_tv = combined_TV_dataset['user_id_encoded'].max() + 1
num_animes_tv = combined_TV_dataset['anime_id_encoded'].max() + 1
num_users_movie = Movie_dataset['user_id_encoded'].max() + 1
num_animes_movie = Movie_dataset['anime_id_encoded'].max() + 1
embedding_size = 128

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Create an instance of the model for tv
model_tv = CollaborativeFilteringModel(num_users_tv, num_animes_tv, embedding_size).to(device)

# Create an instance of the model for movie
model_movie = CollaborativeFilteringModel(num_users_movie, num_animes_movie, embedding_size).to(device)

print(num_users_tv)
print(num_animes_tv)
print(num_users_movie)
print(num_animes_movie)

cuda
268584
12321
183380
2685


In [79]:
# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer_tv = optim.Adam(model_tv.parameters())
optimizer_movie = optim.Adam(model_movie.parameters())

In [83]:
# Training loop for tv model
num_epochs = 20
batch_size = 10000

for epoch in range(num_epochs):
    for i in range(0, len(X_tv_train), batch_size):
        batch_users = torch.LongTensor(X_tv_train[i:i+batch_size, 0]).to(device)
        batch_animes = torch.LongTensor(X_tv_train[i:i+batch_size, 1]).to(device)
        batch_ratings = torch.FloatTensor(y_tv_train[i:i+batch_size]).to(device)

        # Forward pass
        outputs = model_tv(batch_users, batch_animes)
        loss = criterion(outputs.squeeze(), batch_ratings)

        # Backward pass and optimization
        optimizer_tv.zero_grad()
        loss.backward()
        optimizer_tv.step()

    # Print the loss for every epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate the model on the test set
with torch.no_grad():
    test_users = torch.LongTensor(X_tv_test[:, 0]).to(device)
    test_animes = torch.LongTensor(X_tv_test[:, 1]).to(device)
    test_ratings = torch.FloatTensor(y_tv_test).to(device)

    test_outputs = model_tv(test_users, test_animes)
    test_loss = criterion(test_outputs.squeeze(), test_ratings)

print(f"Test Loss: {test_loss.item():.4f}")

Epoch [1/20], Loss: 0.5514
Epoch [2/20], Loss: 0.5502
Epoch [3/20], Loss: 0.5491
Epoch [4/20], Loss: 0.5479
Epoch [5/20], Loss: 0.5467
Epoch [6/20], Loss: 0.5460
Epoch [7/20], Loss: 0.5452
Epoch [8/20], Loss: 0.5446
Epoch [9/20], Loss: 0.5441
Epoch [10/20], Loss: 0.5436
Epoch [11/20], Loss: 0.5429
Epoch [12/20], Loss: 0.5424
Epoch [13/20], Loss: 0.5419
Epoch [14/20], Loss: 0.5414
Epoch [15/20], Loss: 0.5409
Epoch [16/20], Loss: 0.5405
Epoch [17/20], Loss: 0.5400
Epoch [18/20], Loss: 0.5397
Epoch [19/20], Loss: 0.5394
Epoch [20/20], Loss: 0.5391
Test Loss: 0.5781


In [85]:
torch.save(model_tv.state_dict(), 'tv_model_weights.pth')

In [86]:
# Training loop for movie model
num_epochs = 20
batch_size = 10000

for epoch in range(num_epochs):
    for i in range(0, len(X_movie_train), batch_size):
        batch_users = torch.LongTensor(X_movie_train[i:i+batch_size, 0]).to(device)
        batch_animes = torch.LongTensor(X_movie_train[i:i+batch_size, 1]).to(device)
        batch_ratings = torch.FloatTensor(y_movie_train[i:i+batch_size]).to(device)

        # Forward pass
        outputs = model_movie(batch_users, batch_animes)
        loss = criterion(outputs.squeeze(), batch_ratings)

        # Backward pass and optimization
        optimizer_movie.zero_grad()
        loss.backward()
        optimizer_movie.step()

    # Print the loss for every epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate the model on the test set
with torch.no_grad():
    test_users = torch.LongTensor(X_movie_test[:, 0]).to(device)
    test_animes = torch.LongTensor(X_movie_test[:, 1]).to(device)
    test_ratings = torch.FloatTensor(y_movie_test).to(device)

    test_outputs = model_movie(test_users, test_animes)
    test_loss = criterion(test_outputs.squeeze(), test_ratings)

print(f"Test Loss: {test_loss.item():.4f}")

Epoch [1/20], Loss: 0.6169
Epoch [2/20], Loss: 0.5741
Epoch [3/20], Loss: 0.5606
Epoch [4/20], Loss: 0.5570
Epoch [5/20], Loss: 0.5560
Epoch [6/20], Loss: 0.5557
Epoch [7/20], Loss: 0.5555
Epoch [8/20], Loss: 0.5555
Epoch [9/20], Loss: 0.5555
Epoch [10/20], Loss: 0.5555
Epoch [11/20], Loss: 0.5554
Epoch [12/20], Loss: 0.5554
Epoch [13/20], Loss: 0.5551
Epoch [14/20], Loss: 0.5546
Epoch [15/20], Loss: 0.5537
Epoch [16/20], Loss: 0.5527
Epoch [17/20], Loss: 0.5518
Epoch [18/20], Loss: 0.5510
Epoch [19/20], Loss: 0.5504
Epoch [20/20], Loss: 0.5499
Test Loss: 0.5583


In [87]:
torch.save(model_movie.state_dict(), 'movie_model_weights.pth')

In [128]:
# use model to predict
def extract_weights(layer):
    # Get the weights from the PyTorch layer
    weights = layer.weight.data
    
    # Convert to numpy for normalization
    weights = weights = weights.cpu().numpy()
    
    # Normalize the weights
    weights = weights / np.linalg.norm(weights, axis=1, keepdims=True)
    
    return weights

anime_weights_tv = extract_weights(model_tv.anime_embedding)
user_weights_tv = extract_weights(model_tv.user_embedding)
anime_weights_movie = extract_weights(model_movie.anime_embedding)
user_weights_movie = extract_weights(model_movie.user_embedding)

In [129]:
# Grouping the dataset by genre
grouped_anime = df_anime.groupby('Type')

# Creating a dictionary of DataFrames for each tpye
type_anime = {Type: group.copy() for Type, group in grouped_anime}

In [130]:
# seperate the dataset based on Type
TV_anime = type_anime['TV']
Movie_anime = type_anime['Movie']
OVA_anime = type_anime['OVA']
ONA_anime = type_anime['ONA']
Special_anime = type_anime['Special']
Music_anime = type_anime['Music'] # weird type

In [131]:
combined_TV_anime = pd.concat([TV_anime, OVA_anime, Special_anime, ONA_anime]) # include all episodes into tv category

In [153]:
# since the same encoder was used to fit movie again, need to refit to tv
combined_TV_dataset['anime_id_encoded'] = anime_encoder.fit_transform(combined_TV_dataset['anime_id'])
combined_TV_dataset['user_id_encoded'] = user_encoder.fit_transform(combined_TV_dataset['user_id'])

def find_similar_animes_tv(name, n=10, return_dist=False, neg=False):
    try:
        anime_row = combined_TV_anime[combined_TV_anime['Name'].str.lower() == name.lower()].iloc[0]
        #print(anime_row)
        index = anime_row['anime_id']
        #print(index)
        encoded_index = anime_encoder.transform([index])[0]
        weights = anime_weights_tv
        dists = np.dot(weights, weights[encoded_index])
        sorted_dists = np.argsort(dists)
        n = n + 1            
        if neg:
            closest = sorted_dists[:n]
        else:
            closest = sorted_dists[-n:]
        print('Animes closest to {}'.format(name))
        if return_dist:
            return dists, closest
        
        SimilarityArr = []
        
        for close in closest:
            decoded_id = anime_encoder.inverse_transform([close])[0]
            anime_frame = combined_TV_anime[combined_TV_anime['anime_id'] == decoded_id]
            
            anime_name = anime_frame['Name'].values[0]
            english_name = anime_frame['English name'].values[0]
            name = english_name if english_name != "UNKNOWN" else anime_name
            genre = anime_frame['Genres'].values[0]
            Synopsis = anime_frame['Synopsis'].values[0]
            similarity = dists[close]
            similarity = "{:.2f}%".format(similarity * 100)
            SimilarityArr.append({"Name": name, "Similarity": similarity, "Genres": genre, "Synopsis":Synopsis})
        Frame = pd.DataFrame(SimilarityArr).sort_values(by="Similarity", ascending=False)
        return Frame[Frame.Name != name]
    except Exception as e:
        print('{} not found in Anime list. Error: {}'.format(name, str(e)))

pd.set_option('display.max_colwidth', None)

In [154]:
find_similar_animes_tv('Mushoku Tensei: Isekai Ittara Honki Dasu', n=5, neg=False)

Animes closest to Mushoku Tensei: Isekai Ittara Honki Dasu


,Name,Similarity,Genres,Synopsis
4,Mushoku Tensei: Jobless Reincarnation Part 2,48.08%,"Drama, Fantasy, Ecchi","After the mysterious mana calamity, Rudeus Greyrat and his fierce student Eris Boreas Greyrat are teleported to the Demon Continent. There, they team up with their newfound companion Ruijerd Supardia—the former leader of the Superd's Warrior group—to form ""Dead End,"" a successful adventurer party. Making a name for themselves, the trio journeys across the continent to make their way back home to Fittoa.\n\nFollowing the advice he received from the faceless god Hitogami, Rudeus saves Kishirika Kishirisu, the Great Emperor of the Demon World, who rewards him by granting him a strange power. Now, as Rudeus masters the powerful ability that offers a number of new opportunities, it might prove to be more than what he bargained for when unexpected dangers threaten to hinder their travels."
3,Attack on Titan Season 3 Part 2,48.02%,"Action, Drama","Seeking to restore humanity's diminishing hope, the Survey Corps embark on a mission to retake Wall Maria, where the battle against the merciless ""Titans"" takes the stage once again.\n\nReturning to the tattered Shiganshina District that was once his home, Eren Yeager and the Corps find the town oddly unoccupied by Titans. Even after the outer gate is plugged, they strangely encounter no opposition. The mission progresses smoothly until Armin Arlert, highly suspicious of the enemy's absence, discovers distressing signs of a potential scheme against them. \n\nShingeki no Kyojin Season 3 Part 2 follows Eren as he vows to take back everything that was once his. Alongside him, the Survey Corps strive—through countless sacrifices—to carve a path towards victory and uncover the secrets locked away in the Yeager family's basement."
2,Fullmetal Alchemist: Brotherhood,46.15%,"Action, Adventure, Drama, Fantasy","After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse's body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse's soul in the physical realm by binding it to a hulking suit of armor.\n\nThe brothers are rescued by their neighbor Pinako Rockbell and her granddaughter Winry. Known as a bio-mechanical engineering prodigy, Winry creates prosthetic limbs for Edward by utilizing ""automail,"" a tough, versatile metal used in robots and combat armor. After years of training, the Elric brothers set off on a quest to restore their bodies by locating the Philosopher's Stone—a powerful gem that allows an alchemist to defy the traditional laws of Equivalent Exchange.\n\nAs Edward becomes an infamous alchemist and gains the nickname ""Fullmetal,"" the boys' journey embroils them in a growing conspiracy that threatens the fate of the world."
1,Darker than Black,46.13%,"Action, Mystery, Sci-Fi","It has been 10 years since Heaven's Gate appeared in South America and Hell's Gate appeared in Japan, veiling the once familiar night sky with an oppressive skyscape. Their purposes unknown, these Gates are spaces in which the very laws of physics are ignored. With the appearance of the Gates emerged Contractors, who, in exchange for their humanity, are granted supernatural abilities.\n\nIn the Japanese city surrounding Hell’s Gate, Section 4 Chief Misaki Kirihara finds herself at odds with an infamous Contractor codenamed Hei. Called ""Black Reaper"" in the underground world, Hei, like his associates, undertakes missions for the mysterious and ruthless Syndicate while slowly peeling back the dark layers covering a nefarious plot that threatens the very existence of Contractors.\n\nFrom the mind of Tensai Okamura comes a sci-fi thriller taking the form of a subtle exposé on a war in which political po

In [155]:
# since the same encoder was used to fit tv again, need to refit to movie
Movie_dataset['anime_id_encoded'] = anime_encoder.fit_transform(Movie_dataset['anime_id'])
Movie_dataset['user_id_encoded'] = user_encoder.fit_transform(Movie_dataset['user_id'])

def find_similar_animes_movie(name, n=10, return_dist=False, neg=False):
    try:
        anime_row = Movie_anime[Movie_anime['Name'].str.lower() == name.lower()].iloc[0]
        #print(anime_row)
        index = anime_row['anime_id']
        #print(index)
        encoded_index = anime_encoder.transform([index])[0]
        weights = anime_weights_movie
        dists = np.dot(weights, weights[encoded_index])
        sorted_dists = np.argsort(dists)
        n = n + 1            
        if neg:
            closest = sorted_dists[:n]
        else:
            closest = sorted_dists[-n:]
        print('Animes closest to {}'.format(name))
        if return_dist:
            return dists, closest
        
        SimilarityArr = []
        
        for close in closest:
            decoded_id = anime_encoder.inverse_transform([close])[0]
            anime_frame = Movie_anime[Movie_anime['anime_id'] == decoded_id]
            
            anime_name = anime_frame['Name'].values[0]
            english_name = anime_frame['English name'].values[0]
            name = english_name if english_name != "UNKNOWN" else anime_name
            genre = anime_frame['Genres'].values[0]
            Synopsis = anime_frame['Synopsis'].values[0]
            similarity = dists[close]
            similarity = "{:.2f}%".format(similarity * 100)
            SimilarityArr.append({"Name": name, "Similarity": similarity, "Genres": genre, "Synopsis":Synopsis})
        Frame = pd.DataFrame(SimilarityArr).sort_values(by="Similarity", ascending=False)
        return Frame[Frame.Name != name]
    except Exception as e:
        print('{} not found in Anime list. Error: {}'.format(name, str(e)))

pd.set_option('display.max_colwidth', None)

In [156]:
find_similar_animes_movie('Kimi no Na wa.', n=5, neg=False)

Animes closest to Kimi no Na wa.


,Name,Similarity,Genres,Synopsis
4,Space Pirate Captain Harlock,32.15%,"Action, Drama, Sci-Fi","After conquering the galaxy, mankind has reached its apex and is now on a steady decline. The resources of the universe are diminishing, and around 500 billion humans begin to return home to Earth. Thus begins a war between the various factions of humans for control of the planet. Eventually, an authoritarian government known as the Gaia Sanction prevents the re-population of Earth.\n\nCaptain Herlock was one of the Elite Wing who was tasked by the Gaia Sanction to defend the Earth. His fleet of ships was unstoppable under his command. However, the Gaia Sanction allows a diplomatic elite to immigrate to Earth, which angers Herlock. He goes rogue and fires upon the diplomats and the rest of his fleet. He decides to unleash dark matter on the planet to make it uninhabitable, but also becomes engulfed in it which immortalizes him.\n\nOne hundred years have passed, and the legendary Captain Herlock is still at large, with only the Arcadia under his control. The Gaia Sanction continues with their plans for control over the Earth, while hiding its true state. But Captain Herlock is preparing for one final showdown with them, which will determine the fate of the world!"
3,Attack No.1: Namida no Fushichou,29.26%,"Drama, Sports",The fourth and final recap film of Attack No.1. Now set in high school the team competes at inter-high but tragedy strikes.
2,Mobile Suit Gundam I,28.02%,"Action, Adventure, Drama, Sci-Fi","In the future, with Earth's population rapidly increasing, humanity has developed giant orbiting space colonies gathered in clusters known as ""Sides."" As the elites on Earth force most of the populace into space, rebellion begins to brew. In the year Universal Century 0079, one of the colony clusters known as Side 3 declares itself the Principality of Zeon and launches a war of independence against the Earth Federation.\n\nDespite having only a fraction of the resources of the Federation, Zeon quickly gains the upper hand in the conflict through the use of giant humanoid robots called mobile suits. Chief among Zeon's mobile suit ace pilots is Char Aznable, a mysterious masked man known as the ""Red Comet."" When Char's squadron comes across the Federation developing its own prototype mobile suit called Gundam in the colony Side 7, his men launch a brutal attack. Amuro Ray, the son of the prototype's developer, gets in the Gundam and is able to fend off the Zeon attack. As Amuro soon finds himself directly in conflict with Char, he is forced to flee the colony on a Federation ship."
1,Gundam Neo Experience 0087: Green Divers,26.26%,Adventure,"During the closing days of the Gryps conflict of Mobile Suit Zeta Gundam, two siblings—Asagi and Takuya—ride aboard a civilian transport bound for Earth. Caught in a battle between the Titans and AEUG, they must somehow guide their ship through atmospheric entry. Luckily, they're not completely alone, as they find aid from a custom-painted Zeta Gundam, whose pilot sounds a lot like One Year War ace Amuro Ray.\n\n(Source: ANN)"
0,Arabian Nights: Sinbad's Adventures,26.19%,"Action, Adventure, Fantasy","This is a work by Toei Animation Co., Ltd. with screenplay by Tezuka Osamu in cooperation with novelist Kita Morio. Sinbad and a boy, Ali, are stowaways on the transport ship ""Boulder,"" but they win their proper places on the ship after currying the favor of the captain. \nThen the beautiful princess Samir jumps on board to escape a wicked minister who wants to marry her, and Sinbad goes through a series of adventures to protect the princess. \n\nThe story unfolds with ups and downs, full of dramatic episodes from the original story such as a rough-and-tumble battle with a strange bird that is guarding diamonds."
